In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pickle

In [2]:
# Load ALREADY-SPLIT datasets (SOURCE OF TRUTH)
train_df = pd.read_csv(r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\squat_model/data\squat_train_keypoints.csv")
test_df  = pd.read_csv(r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\squat_model/data\squat_test_keypoints.csv")

X_train = train_df.drop("label", axis=1).values.astype(np.float32)
y_train = train_df["label"].values.astype(np.float32)

X_test = test_df.drop("label", axis=1).values.astype(np.float32)
y_test = test_df["label"].values.astype(np.float32)

# Fit scaler ONLY on TRAIN
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

# Save scaler for live
with open(r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\squat_model\model\scaler_keypoints.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Torch tensors
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train)

X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader  = DataLoader(TensorDataset(X_test, y_test), batch_size=32)

# SANITY CHECK
print("TRAIN:", X_train.shape, "TEST:", X_test.shape)
print("Label distrib train:", np.unique(y_train.numpy(), return_counts=True))
print("Label distrib test :", np.unique(y_test.numpy(), return_counts=True))

TRAIN: torch.Size([2942, 68]) TEST: torch.Size([1511, 68])
Label distrib train: (array([0., 1.], dtype=float32), array([1695, 1247], dtype=int64))
Label distrib test : (array([0., 1.], dtype=float32), array([ 475, 1036], dtype=int64))


In [3]:
class MLP_Keypoints(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),

            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

model = MLP_Keypoints(input_dim=X_train.shape[1])

In [14]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [15]:
EPOCHS = 20

for epoch in range(EPOCHS):

    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        preds = model(xb).squeeze()
        loss = loss_fn(preds, yb)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {total_loss/len(train_loader):.4f}")


Epoch 1/20 - Loss: 0.0002
Epoch 2/20 - Loss: 0.0085
Epoch 3/20 - Loss: 0.0058
Epoch 4/20 - Loss: 0.0094
Epoch 5/20 - Loss: 0.0104
Epoch 6/20 - Loss: 0.0034
Epoch 7/20 - Loss: 0.0041
Epoch 8/20 - Loss: 0.0015
Epoch 9/20 - Loss: 0.0029
Epoch 10/20 - Loss: 0.0028
Epoch 11/20 - Loss: 0.0007
Epoch 12/20 - Loss: 0.0002
Epoch 13/20 - Loss: 0.0001
Epoch 14/20 - Loss: 0.0001
Epoch 15/20 - Loss: 0.0001
Epoch 16/20 - Loss: 0.0000
Epoch 17/20 - Loss: 0.0000
Epoch 18/20 - Loss: 0.0003
Epoch 19/20 - Loss: 0.0021
Epoch 20/20 - Loss: 0.0402


In [16]:
model.eval()
with torch.no_grad():
    preds = model(X_test).squeeze()
    preds = torch.sigmoid(preds)
    preds = (preds > 0.5).float()

accuracy = (preds == y_test).float().mean()
print("Accuracy:", accuracy.item())

Accuracy: 0.8881535530090332


In [17]:
EPOCHS = 15
train_losses = []
test_losses = []
best_loss = float("inf")
patience = 2
wait = 0

for epoch in range(EPOCHS):

    model.train()
    total_loss = 0

    for xb, yb in train_loader:
        preds = model(xb).squeeze()
        loss = loss_fn(preds, yb)

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        total_loss += loss.item()

    train_losses.append(total_loss / len(train_loader))

    # Validation
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            preds = model(xb).squeeze()
            loss = loss_fn(preds, yb)
            val_loss += loss.item()

    val_loss /= len(test_loader)
    test_losses.append(val_loss)

    print(f"Epoch {epoch+1}/{EPOCHS} - Train Loss: {train_losses[-1]:.4f} - Test Loss: {val_loss:.4f}")

Epoch 1/15 - Train Loss: 0.0036 - Test Loss: 0.6991
Epoch 2/15 - Train Loss: 0.0005 - Test Loss: 0.7399
Epoch 3/15 - Train Loss: 0.0009 - Test Loss: 0.5982
Epoch 4/15 - Train Loss: 0.0004 - Test Loss: 0.6060
Epoch 5/15 - Train Loss: 0.0002 - Test Loss: 0.6750
Epoch 6/15 - Train Loss: 0.0003 - Test Loss: 0.4828
Epoch 7/15 - Train Loss: 0.0029 - Test Loss: 0.6965
Epoch 8/15 - Train Loss: 0.0011 - Test Loss: 0.6669
Epoch 9/15 - Train Loss: 0.0070 - Test Loss: 0.5233
Epoch 10/15 - Train Loss: 0.0031 - Test Loss: 0.5687
Epoch 11/15 - Train Loss: 0.0005 - Test Loss: 0.8366
Epoch 12/15 - Train Loss: 0.0003 - Test Loss: 0.6583
Epoch 13/15 - Train Loss: 0.0002 - Test Loss: 0.8219
Epoch 14/15 - Train Loss: 0.0011 - Test Loss: 1.0189
Epoch 15/15 - Train Loss: 0.0003 - Test Loss: 0.8617


In [18]:
torch.save(model.state_dict(), r"C:\Users\caovi\OneDrive\Desktop\projet annuel\core\squat_model\model\squat_mlp_keypoints.pt")
print("Modèle KEYPOINTS sauvegardé.")

Modèle KEYPOINTS sauvegardé.


In [19]:
model.eval()
with torch.no_grad():
    preds = torch.sigmoid(model(X_test)).squeeze()
    preds = (preds > 0.5).float()
    acc = (preds == y_test).float().mean().item()

print("FINAL TEST ACCURACY:", acc)

FINAL TEST ACCURACY: 0.8987425565719604
